In [94]:
#from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import haversine
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [103]:
spark_ = SparkSession.builder.appName('practice').getOrCreate()

In [104]:
df = spark_.read.option('header', True).csv('go_bike_with_distance.csv', inferSchema=True)
# uncal_data = spark_.read.option('header', True).csv('tripdata.csv', inferSchema=True)

In [111]:
#df = df.withColumnRenamed('distance', 'cal_distant')
df = df.drop('_c0') # removing unnecceray column

In [113]:
df.show(3) # view is not clear

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+-----------------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|user_type|member_birth_year|member_gender|     pyment|Distance_in_meter|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+-----------------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96| Customer

tasks for this week : 
1-calculate the total distance for each bike and list the top 10
2-calculate the number of trips for each start station list top 10 and find the ratio of using as male
            or female
3-make a comparison to find the percentage of usage for customer and subscriber
4-calculate the age of all users and show the relation between the distance and the age
team leader : moihbur 
!! siraj will send the dataset with distance column !!

### 1-calculate the total distance for each bike and list the top 10

In [115]:
df.show(n=3,vertical=True) # make a clear view. RECORD repressnts row number

-RECORD 0---------------------------------------
 start_time              | 57:39.7              
 end_time                | 12:50.2              
 start_station_id        | 74                   
 start_station_name      | Laguna St at Haye... 
 start_station_latitude  | 37.77643482          
 start_station_longitude | -122.426244          
 end_station_id          | 43                   
 end_station_name        | San Francisco Pub... 
 end_station_latitude    | 37.7787677           
 end_station_longitude   | -122.4159292         
 bike_id                 | 96                   
 user_type               | Customer             
 member_birth_year       | 1987                 
 member_gender           | Male                 
 pyment                  | credit card          
 Distance_in_meter       | 942.93               
-RECORD 1---------------------------------------
 start_time              | 56:34.8              
 end_time                | 49:55.6              
 start_station_id   

In [117]:
df = df.withColumnRenamed('Distance_in_meter', 'dist_meter') # simply renaming column

In [118]:
df.select('bike_id').distinct().count() # checking how many unique values in bike_id -> 3673

3673

In [119]:
bike_distance = df.groupBy('bike_id').agg({'dist_meter': 'sum'}) # total distant for each bike

In [121]:
bike_distance.count() # same as before -> 3673

bike_distance.show()

+-------+------------------+
|bike_id|   sum(dist_meter)|
+-------+------------------+
|   2366|         249465.29|
|   2659|         490692.41|
|    496|         396983.32|
|    471| 578013.0599999999|
|   1238|167757.93999999997|
|   1591|415585.58999999997|
|    148|210349.24999999994|
|   2122|         185749.79|
|   2866|403550.13999999996|
|   1580|226680.16999999993|
|   1645| 328620.4600000001|
|   1829|415100.30999999994|
|   1342|167125.82000000004|
|   2142|         162044.23|
|    833|15854.170000000002|
|    463|         108842.98|
|   1959|219682.59999999998|
|   1088|         523288.28|
|   3175|         258649.06|
|   2563|         196767.43|
+-------+------------------+
only showing top 20 rows



In [127]:
bike_distance = bike_distance.withColumnRenamed('sum(dist_meter)', 'bike_distance') # renaming column

In [128]:
bike_distance.sort(desc('bike_distance')).show(n=10)  # top 10 bikers with id usinf sort function

+-------+-----------------+
|bike_id|    bike_distance|
+-------+-----------------+
|     68|743369.3899999999|
|   2178|721182.2200000001|
|    256|         671916.0|
|    235|        670162.09|
|   2049|        656828.08|
|    441|        656642.33|
|   2226|647823.2399999999|
|    796|646867.8099999998|
|    190|640294.2399999999|
|   2365|         639412.8|
+-------+-----------------+
only showing top 10 rows



### 2-calculate the number of trips for each start station list top 10 and find the ratio of using as male or female
#### calculate the number of trips for each start station list top 10
#### find the ratio of using as male or female

In [164]:
""""wild card charecter (*) referes every column in dataset. In this case, * apple the count on start_station_name column.
# simple help(df.groupBy) will show how this function works"""

count_start_station = df.groupBy('start_station_name',)\
.agg({'*': 'count'}) # calculate the number of trips for each start station list top 10

In [165]:
# possible to change the column name
count_start_station.sort(desc('count(1)')).show(n=10) # list top 10

+--------------------+--------+
|  start_station_name|count(1)|
+--------------------+--------+
|San Francisco Fer...|   15187|
|The Embarcadero a...|   13664|
|San Francisco Cal...|   12546|
|San Francisco Cal...|   12055|
|Market St at 10th St|   11960|
|Montgomery St BAR...|   11334|
|  Berry St at 4th St|   10956|
|Powell St BART St...|   10142|
|Howard St at Beal...|    9926|
|Steuart St at Mar...|    9347|
+--------------------+--------+
only showing top 10 rows



In [157]:
ratio_gender = df.groupBy('start_station_name','member_gender' )\
.agg({'*': 'count'})

In [176]:
ratio_gender.where((df.member_gender == 'Male') | (df.member_gender =='Female')).show()

+--------------------+-------------+--------+
|  start_station_name|member_gender|count(1)|
+--------------------+-------------+--------+
| George St at 1st St|         Male|     100|
|Shattuck Ave at 5...|         Male|     251|
| Julian St at 6th St|       Female|      25|
|Laguna St at Haye...|         Male|    1840|
|14th St at Filber...|         Male|      18|
|Grand Ave at Perk...|         Male|    1736|
|      DeFremery Park|       Female|     169|
|14th St at Missio...|         Male|    1556|
|Valencia St at 22...|         Male|    1397|
| 5th St at Taylor St|         Male|      77|
|Fruitvale Ave at ...|         Male|      50|
|Telegraph Ave at ...|       Female|     517|
|29th St at Tiffan...|       Female|     381|
|Stanford Ave at H...|       Female|      39|
|49th St at Telegr...|       Female|     239|
|Julian St at The ...|         Male|     540|
|Oregon St at Adel...|         Male|     167|
| Page St at Scott St|         Male|    1774|
|The Embarcadero a...|         Mal

In [178]:
df.groupBy('member_gender').agg({'start_station_name', 'sum'})

AssertionError: all exprs should be Column